# Test Raspi GPio et I2C  Image ENS-Lyon Ife n-1


L'image utilisée a été modifiée pour permettre l'utilisation du GPIO. La prochaine mise à jour de l'image intégrera ce réglage.
L'image utilisée nécessite le lancement de jupyterhub en tant que root, ce mde de lancement sera inutile dans la prochaine mise à jour de l'image.

Le cahier présente des situations variées pour explorer le fonctionnement des LED simples et RGB à Anode commune, ainsi que le moyen de récupérer la température du capteur mcp9808.

L'appropriation de ces situations est proposée en couplant l'allumage des LEDs au franchissement d'un ou de plusieurs seuils de température.

Un prologement pourrait être l'utilisatio d'un capteur d'humidité et l'affichage chromatique des palliers d'humidité relative.

Le bredboard utilisé se présente comme sur l'illustration ci-dessous

![BredBoard avec l'ensemble des outils mis en place](images/BredBoardTestTempLeds.jpg)
                                
![licence](images/Licence.jpg)
    

# Un premier test sur mcp9808

On importe les librairies nécessaires puis on décide de choisir le nombre de mesure à effectuer et le délai entre deux mesures.

In [1]:
import mraa, upm
import time
from upm import pyupm_mcp9808 as mcp
mcp = mcp.MCP9808(0)


In [2]:
tempMCP = mcp.getTemp()
print ("{0:.2f} °C".format(tempMCP)," ;")

24.62 °C  ;


In [3]:
nombremesures=int(input("Saisissez le nombre de mesures souhaitées : "))
cycle=0
pause=int(input("Saisissez l'intervalle entre 2 mesures en secondes : "))
while cycle < nombremesures:
    tempMCP = mcp.getTemp()
    manufacturer=mcp.getManufacturer()
    hysteresis=mcp.getHysteresis()
    print ("{0:.2f} °C".format(tempMCP)," ;")
    time.sleep(pause)
    cycle=cycle+1
print('Code fabricant : {0:d} ; Hysteresis {1:.2f} ;'.format(manufacturer,hysteresis))

Saisissez le nombre de mesures souhaitées : 10
Saisissez l'intervalle entre 2 mesures en secondes : 1
24.62 °C  ;
24.62 °C  ;
24.62 °C  ;
24.62 °C  ;
24.62 °C  ;
24.62 °C  ;
24.62 °C  ;
24.62 °C  ;
24.62 °C  ;
24.56 °C  ;
Code fabricant : 84 ; Hysteresis 0.00 ;


# Le capteur mcp9808 et la récupération de la Température.

Ce programme plus élaboré est issu du Cahier de Programmes conçu par Gérard Vidal dans le cadre du PIA "Météorologie et climat : Tremplin pour l'enseignement des sciences".
Il présente une utilisation des fonctions de Python.

In [14]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# Imports from mraa and upm libraries
#------------------------------------
import mraa, upm
import time, math
import _thread

from upm import pyupm_mcp9808

# Constant declarations
#----------------------
# i2c bus 0 on RaspBerry Pi 
bus1 = 0
mcpAddr = 0x60
UPPER_TEMP = 0x02
LOWER_TEMP = 0x03
CRIT_TEMP = 0x04


# create sensors on the bus 1
mcp = pyupm_mcp9808.MCP9808(bus1)
#mcp = pyupm_mcp9808.MCP9808(bus1, mcpAddr)


In [15]:
#Functions
#---------
# MCP9808
#********
def get_mcpSensor():
	unity = mcp.isCelsius()
	upper = mcp.getMonitorReg (UPPER_TEMP)
	lower = mcp.getMonitorReg (LOWER_TEMP)
	critic = mcp.getMonitorReg (CRIT_TEMP)
	hyst = mcp.getHysteresis ()
	resol = mcp.getResolution ()
	idManuf = mcp.getManufacturer ()
	mcpId = mcp.getDevicedId ()
	if unity :
		print ('We are measuring in Celsius')
	else :
		print ('We are measuring in Fahrenheit')
	print ('\tLower T° : %.3f\n'%lower + '\tUpperT° : %.3f\n'%upper + '\tCritical T° : %.3f\n'%critic)
	print ('Hysteresis value : %.3f'%hyst)
	print ('Resolution : %.4f'%resol)
	print ('Manufacturer Id : %d'%idManuf)
	print ('Device Id : %d\n'%mcpId)
	return
	
def mes_mcpTemp():
	mcpTemp = mcp.getTemp ()
	return (mcpTemp)

#Activate or test some sensors
#---------------------
#mcp.shutDown(False)

In [16]:
# Main function
###############	
def input_thread(CheckInput):
	check = input()
	CheckInput.append(check)
	return
	
def init_Thread () :
	CheckInput = []
	_thread.start_new_thread(input_thread, (CheckInput,))
	print ('Instructions :  \t"stop" pour arrêter les mesures.\n')
	return (CheckInput)
	   
def measureMeteoParams():
	nbmes = 0
	while True :
		option1 = input('Instructions :  \t"start" pour démarrer les mesures "check" pour la maintenance "exit" pour sortir.\n--> ')
		if option1 == 'exit' :
			break
		elif option1 == 'start' :
			break
		elif option1 == 'check' :
			option2 = 'ready'
			while option2.strip != exit :
				option2 = input('Taper "info" pour avoir des informations, "changer" pour modifier les paramètres des capteurs, "exit" pour sortir.\n---> ')
				if option2 == 'exit':
					break
				elif option2 == 'info':
					option3 = 4
					while option3 != 0 :
						option3 = input('Informations sur les capteurs : MCP9808 (1) exit (0)\n----> ')
						if option3 == '0' :
							break
						elif option3 == '1' :
							get_mcpSensor()
						else :
							print ('Code inconnu : %s'%option3)
				elif option2 == 'changer' :
					option3 = 3
					while option3 != 0 :
						option3 = input('Changement des paramètres des capteurs :\n\tMCP9808-precision (1)\n\tMCP9808-hysteresis (2)\n\t sortir (0)\n----> ')
						if option3 == '0' :
							break
						elif option3 == '1' :
							val = input("precision value (LOW MED HI MAX) : ")
							if val == 'LOW' :
								mcp.setResolution (0x00)
							elif val == 'MED' :
								mcp.setResolution (0x01)
							elif val == 'HI' :
								mcp.setResolution (0x02)
							elif val == 'MAX' :
								mcp.setResolution (0x03)
							else :
								print ('Commande inconnue : "%s"\n'%val)
						elif option3 == '2' :
							val = input("Hysteresis value  (HYST_0 HYST_1_5 HYST_3_0 HYST_6_0) : ")
							if val == 'HYST_0' :
								mcp.setHysteresis (0x0000)
							elif val == 'HYST_1_5' :
								mcp.setHysteresis (0x0002)
							elif val == 'HYST_3_0' :
								mcp.setHysteresis (0x0004)
							elif val == 'HYST_6_0' :
								mcp.setHysteresis (0x0006)
							else :
								print ('Commande inconnue : "%s"\n'%val)
						else :
							print ('Code inconnu : "%s"\n'%option3)
				else :
					print ('Commande inconnue : "%s"\n'%option2)
		else :
			print ('Commande inconnue : "%s"\n'%option1)
# Main loop
	if option1.strip() == 'start' :
		CheckInput = init_Thread()
		while True :
			if CheckInput == ['stop']: 
				print ('Arrêt demandé.')
				break
			elif CheckInput != [] :
				print ('Commande inconnue : "%s"\n'%CheckInput)
				thread.start_new_thread(input_thread, (CheckInput,))
				del CheckInput[:]
			else :
				time.sleep (0.5)
			tempHR = mcp.getTemp()
			if nbmes == 0 :
				print ('Temp(MCP) : ') 
				nbmes += 1
			elif nbmes == 9 :
				nbmes = 0
			else :
				nbmes += 1
			print ('%.4f °C\t' %tempHR)
			
			time.sleep (5)
	else :
		print ('Bye...')
	return

measureMeteoParams()

Instructions :  	"start" pour démarrer les mesures "check" pour la maintenance "exit" pour sortir.
--> start
Instructions :  	"stop" pour arrêter les mesures.

Temp(MCP) : 
24.4375 °C	
24.5000 °C	
stop
Arrêt demandé.


# Allumage de deux leds 

Ce programme vous permet de comprendre comment on définit les états de ports GPIO pour qu'ils puissent allumer une led pendant un temps donné.

In [7]:
import mraa, time

# LEDs pilotées par la broche 32 verte et la broche 33 rouge
ledgreen = mraa.Gpio(38)
ledred = mraa.Gpio(40)
# Broches positionnées en sortie
ledgreen.dir(mraa.DIR_OUT)
ledred.dir(mraa.DIR_OUT)

7

In [5]:
# Extinction des LEDs
ledgreen.write(0)
ledred.write(0)
print ("LED Verte allumée 5 secondes")
ledgreen.write(1)
time.sleep(5)
ledgreen.write(0)
print ("LED Rouge allumée 5 secondes")
ledred.write(1)
time.sleep(5)
print("LEDs éteintes")
ledred.write(0)

LED Verte allumée 5 secondes
LED Rouge allumée 5 secondes
LEDs éteintes


99

# Alerte dépassement de température

Ce programme vous permet d'allumer une led verte tant que la température est inférieure à 25 °C et une led rouge si la température dépasse 25 °C

Repérez les déclarations liées au GPIO qui pilote vos leds rouge et verte. La nomenclature des ports du GPIO a été choisie pour correspondre à ce qui est écrit sur le T-Cobbler plus.

In [16]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import time
import mraa, upm
print("Alerte rouge si la température dépasse 25 °C")
from upm import pyupm_mcp9808 as mcp
mcp = mcp.MCP9808(0)
# LEDs pilotées par la broche 32 verte et la broche 33 rouge
ledg = mraa.Gpio(32)
ledr = mraa.Gpio(33)
# Broches positionnées en sortie
ledg.dir(mraa.DIR_OUT)
ledr.dir(mraa.DIR_OUT)
#bme.update()
nombremesures=int(input("Saisissez le nombre de mesures souhaitées : "))
cycle=0
pause=int(input("Saisissez l'intervalle entre 2 mesures en secondes : "))
while cycle < nombremesures:
    tempMCP = mcp.getTemp()
    if tempMCP < 25:
        ledg.write(1)
        ledr.write(0)
    else:
        ledg.write(0)
        ledr.write(1)
    manufacturer=mcp.getManufacturer()
    hysteresis=mcp.getHysteresis()
    print ("{0:.2f} °C".format(tempMCP)," ; ",end='')
    time.sleep(pause)
    cycle=cycle+1

#print(manufacturer," ; ",end='')
#print(hysteresis," ; ",end='')
ledg.write(0)
ledr.write(0)

Alerte rouge si la température dépasse 25 °C
Saisissez le nombre de mesures souhaitées : 100
Saisissez l'intervalle entre 2 mesures en secondes : 1
24.31 °C  ; 24.38 °C  ; 24.31 °C  ; 24.31 °C  ; 24.31 °C  ; 24.31 °C  ; 24.31 °C  ; 24.44 °C  ; 24.50 °C  ; 24.50 °C  ; 24.44 °C  ; 24.56 °C  ; 24.50 °C  ; 24.50 °C  ; 24.50 °C  ; 24.56 °C  ; 24.56 °C  ; 24.50 °C  ; 24.56 °C  ; 24.50 °C  ; 24.50 °C  ; 24.56 °C  ; 24.50 °C  ; 24.50 °C  ; 24.50 °C  ; 24.50 °C  ; 24.50 °C  ; 24.50 °C  ; 24.50 °C  ; 24.50 °C  ; 24.50 °C  ; 

KeyboardInterrupt: 

# La LED RGB anode commune : Une tension nulle quand le GPIO est à HIGH

Pour travailler avec une seule led pour indiquer l'alerte on peut utiliser une LED RVB cela enrichira aussi les possibilités d'illustration des valeurs de température.

Voyons d'abord le programme qui démontre que le GPIO à LOW allume les LEDs Verte puis Rouge

In [4]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import mraa, time
ledg = mraa.Gpio(20)
ledr = mraa.Gpio(21)
ledb = mraa.Gpio(22)
# Broches positionnées en sortie
ledg.dir(mraa.DIR_OUT)
ledr.dir(mraa.DIR_OUT)
ledr.dir(mraa.DIR_OUT)
print("Led Verte et Rouge de la RGB éteintes 2 secondes")
ledg.write(1)
ledr.write(1)
time.sleep(2)
print("Led Verte de la RGB allumée 5 secondes")
ledg.write(0)
time.sleep(5)
print("Led Rouge de la RGB allumée 5 secondes")
ledr.write(0)
ledg.write(1)
time.sleep(5)
print("Led Verte et Rouge de la RGB éteintes")
ledr.write(1)

Led Verte et Rouge de la RGB éteintes 2 secondes
Led Verte de la RGB allumée 5 secondes
Led Rouge de la RGB allumée 5 secondes
Led Verte et Rouge de la RGB éteintes


# Alerte dépassement de température

Cette fois le programme doit vous permettre d'allumer la led verte de la Led RVB  tant que la température est inférieure à 25 °C et la led rouge si la température dépasse 25 °C

In [5]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import RPi.GPIO as GPIO
import time
import mraa, upm
print("Alerte rouge si la température dépasse 25 °C")
from upm import pyupm_mcp9808 as mcp
mcp = mcp.MCP9808(0)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)
GPIO.setup(20,GPIO.OUT)
GPIO.setup(21,GPIO.OUT)
# mise à l'extinction des deux leds
GPIO.output(21,GPIO.HIGH)
GPIO.output(20,GPIO.HIGH)
#bme.update()
nombremesures=int(input("Saisissez le nombre de mesures souhaitées : "))
cycle=0
pause=float(input("Saisissez l'intervalle entre 2 mesures en secondes : "))
while cycle < nombremesures:
    tempMCP = mcp.getTemp()
    if tempMCP < 25:
        GPIO.output(20,GPIO.LOW)
        GPIO.output(21,GPIO.HIGH)
    else:
        GPIO.output(21,GPIO.LOW)
        GPIO.output(20,GPIO.HIGH)
    manufacturer=mcp.getManufacturer()
    hysteresis=mcp.getHysteresis()
    print ("{0:.2f} °C".format(tempMCP)," ; ",end='')
    time.sleep(pause)
    cycle=cycle+1

#print(manufacturer," ; ",end='')
#print(hysteresis," ; ",end='')
GPIO.output(21,GPIO.HIGH)
GPIO.output(20,GPIO.HIGH)

Alerte rouge si la température dépasse 25 °C
Saisissez le nombre de mesures souhaitées : 16
Saisissez l'intervalle entre 2 mesures en secondes : 1.5
19.94 °C  ; 19.94 °C  ; 20.38 °C  ; 25.12 °C  ; 25.94 °C  ; 24.19 °C  ; 22.69 °C  ; 22.06 °C  ; 21.75 °C  ; 21.50 °C  ; 21.38 °C  ; 21.31 °C  ; 21.12 °C  ; 21.06 °C  ; 21.00 °C  ; 20.94 °C  ; 

# La gestion des couleurs avec la led RGB

Pour pouvoir créer un indicateur de température à 3 états on peut utiliser la led RGB et coder par exemple 

- Vert pour T ≤ 22 °C
- Orange pour 22 °C < T ≤ 25 °C
- Rouge pour 25 °C < T

In [8]:
import RPi.GPIO as gpio, time
gpio.setmode(gpio.BCM)  # choisir BCM ou BOARD pour identifier les broches. Ici BOARD
gpio.setwarnings(False)

gpio.setup(24, gpio.OUT)# GPIO 22 broche 15 output  
gpio.setup(20, gpio.OUT)# GPIO 23 broche 16 output 
gpio.setup(21, gpio.OUT)# GPIO 24 broche 18 output 


pb = gpio.PWM(24, 50)    # led bleue crée un objet pg  PWM sur le port GPIO22 à 50 Hertz  T=20ms 
pg = gpio.PWM(21, 50)    # led verte crée un objet pb  PWM sur le port GPIO23 à 50 Hertz  T=20ms 
pr = gpio.PWM(20, 50)    # led rouge crée un objet pr  PWM sur le port GPIO24 à 50 Hertz  T=20ms 

loadr = 0
loadg = 30
loadb = 100
print("Orange : La température est entre 22 °C et 25 °C")
pr.start(loadr)             # allumage à load % (load% duty cycle) 
pg.start(loadg)             # allumage à load % (load% duty cycle)   
pb.start(loadb)             # allumage à load % (load% duty cycle)   

time.sleep(5)

loadr = 0
loadg = 100
loadb = 100
print("Rouge : La température est supérieure ou égale à 25 °C")
pr.start(loadr)             # allumage à load % (load% duty cycle) 
pg.start(loadg)             # allumage à load % (load% duty cycle)   
pb.start(loadb)             # allumage à load % (load% duty cycle)   

time.sleep(5)

loadr = 100
loadg = 0
loadb = 100
print("Vert : La température est inférieure ou égale à 22 °C")
pr.start(loadr)             # allumage à load % (load% duty cycle) 
pg.start(loadg)             # allumage à load % (load% duty cycle)   
pb.start(loadb)             # allumage à load % (load% duty cycle) 

time.sleep(5)

load = 100 

pr.start(load)             # allumage à load % (load% duty cycle) 
pg.start(load)             # allumage à load % (load% duty cycle)   
pb.start(load)             # allumage à load % (load% duty cycle)  

gpio.cleanup()

Orange : La température est entre 22 °C et 25 °C
Rouge : La température est supérieure ou égale à 25 °C
Vert : La température est inférieure ou égale à 22 °C


# Un indicateur de température ambiante 

On utilise le capteur mpc9808 pour lire la température au voisinnage du capteur, on réutilise les données du programme de gamme de couleur pour le coupler à une condition de valeur de la température.


In [9]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# On importe l'ensemble des libraries utiles dans le programme
import RPi.GPIO as GPIO
import time
import mraa, upm
print("Température ambiante : Repère de confort thermique")
# On renomme la librairie pyupm_mcp9808 pour faciliter l'écriture du programme
from upm import pyupm_mcp9808 as mcp
# on déclare le port I2C au Numéro 0
mcp = mcp.MCP9808(0)
# on défini le mode de codage du GPIO 
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)
GPIO.setup(24, GPIO.OUT)# GPIO 24 broche 24 output  
GPIO.setup(20, GPIO.OUT)# GPIO 20 broche 20 output 
GPIO.setup(21, GPIO.OUT)# GPIO 21 broche 21 output
# Création des objets R G B de la led
pb = GPIO.PWM(24, 50)    # led bleue crée un objet pg  PWM sur le port GPIO24 à 50 Hertz  T=20ms 
pg = GPIO.PWM(21, 50)    # led verte crée un objet pb  PWM sur le port GPIO21 à 50 Hertz  T=20ms 
pr = GPIO.PWM(20, 50)    # led rouge crée un objet pr  PWM sur le port GPIO20 à 50 Hertz  T=20ms 
# Mise à 0 de la led RGB anode commune
load=100
pr.start(load)             # extinction à load % (load% duty cycle) 
pg.start(load)             # extinction à load % (load% duty cycle)   
pb.start(load)             # extinction à load % (load% duty cycle)  
loadr=load
loadg=load
loadb=load

nombremesures=int(input("Saisissez le nombre de mesures souhaitées : "))
cycle=0
pause=float(input("Saisissez l'intervalle entre 2 mesures en secondes : "))

while cycle < nombremesures:
    tempMCP = mcp.getTemp()
    if tempMCP < 22:
        loadr = 100
        loadg = 0
        loadb = 100
        pr.start(loadr)             # allumage à load % (load% duty cycle) 
        pg.start(loadg)             # allumage à load % (load% duty cycle)   
        pb.start(loadb)             # allumage à load % (load% duty cycle) 
    elif tempMCP >= 25:
        loadr = 0
        loadg = 100
        loadb = 100
        pr.start(loadr)             # allumage à load % (load% duty cycle) 
        pg.start(loadg)             # allumage à load % (load% duty cycle)   
        pb.start(loadb)             # allumage à load % (load% duty cycle) 
    else:
        loadr = 0
        loadg = 30
        loadb = 100
        pr.start(loadr)             # allumage à load % (load% duty cycle) 
        pg.start(loadg)             # allumage à load % (load% duty cycle)   
        pb.start(loadb)             # allumage à load % (load% duty cycle)   

    manufacturer=mcp.getManufacturer()
    hysteresis=mcp.getHysteresis()
    print ("{0:.2f} °C".format(tempMCP)," ; ",end='')
    time.sleep(pause)
    cycle=cycle+1
load=100
pr.start(load)             # extinction à load % (load% duty cycle) 
pg.start(load)             # extinction à load % (load% duty cycle)   
pb.start(load)             # extinction à load % (load% duty cycle)  

GPIO.cleanup()

Température ambiante : Repère de confort thermique
Saisissez le nombre de mesures souhaitées : 25
Saisissez l'intervalle entre 2 mesures en secondes : 2
20.00 °C  ; 20.00 °C  ; 20.00 °C  ; 23.81 °C  ; 25.44 °C  ; 25.94 °C  ; 26.25 °C  ; 26.44 °C  ; 26.56 °C  ; 26.56 °C  ; 24.25 °C  ; 23.38 °C  ; 22.94 °C  ; 22.62 °C  ; 22.44 °C  ; 22.25 °C  ; 22.12 °C  ; 22.06 °C  ; 21.94 °C  ; 21.88 °C  ; 21.75 °C  ; 21.62 °C  ; 21.56 °C  ; 21.50 °C  ; 21.38 °C  ; 